In [ ]:
# Import libraries
from flask import Flask, request, jsonify
import sqlite3
from datetime import datetime
import pandas as pd
import streamlit as st
import plotly.express as px
import os
import subprocess
import sys

# Check and install missing dependencies
def install_dependencies():
    required_libraries = ['flask', 'streamlit', 'pandas', 'plotly']
    for lib in required_libraries:
        try:
            __import__(lib)
        except ImportError:
            print(f"Installing {lib}...")
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', lib])

# Initialize Flask app
app = Flask(__name__)

# Initialize SQLite database
def init_db():
    conn = sqlite3.connect('chatbot_analytics.db', check_same_thread=False)
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS queries
                 (id INTEGER PRIMARY KEY, user_id TEXT, query_text TEXT, response_text TEXT, timestamp TEXT, satisfaction_rating INTEGER)''')
    conn.commit()
    return conn

# Log a query
@app.route('/log_query', methods=['POST'])
def log_query():
    try:
        data = request.json
        if not data:
            return jsonify({"status": "error", "message": "No data provided"}), 400

        conn = init_db()
        c = conn.cursor()
        c.execute('''INSERT INTO queries (user_id, query_text, response_text, timestamp, satisfaction_rating)
                     VALUES (?, ?, ?, ?, ?)''',
                  (data.get('user_id'), data.get('query_text'), data.get('response_text'), datetime.now(), data.get('satisfaction_rating')))
        conn.commit()
        conn.close()
        return jsonify({"status": "success"}), 200
    except Exception as e:
        return jsonify({"status": "error", "message": str(e)}), 500

# Streamlit Dashboard
def run_dashboard():
    st.title("Chatbot Analytics Dashboard")

    # Connect to the database
    try:
        conn = sqlite3.connect('chatbot_analytics.db', check_same_thread=False)
        queries = pd.read_sql('SELECT * FROM queries', conn)
        queries['timestamp'] = pd.to_datetime(queries['timestamp'])
    except Exception as e:
        st.error(f"Error loading data: {e}")
        return

    # Display total queries
    st.metric("Total Queries", len(queries))

    # Queries over time
    st.subheader("Queries Over Time")
    try:
        queries_over_time = queries.resample('D', on='timestamp').size().reset_index(name='count')
        fig1 = px.line(queries_over_time, x='timestamp', y='count', title="Queries Over Time")
        st.plotly_chart(fig1)
    except Exception as e:
        st.error(f"Error generating Queries Over Time chart: {e}")

    # Most common topics
    st.subheader("Most Common Topics")
    try:
        topics = queries['query_text'].value_counts().reset_index()
        topics.columns = ['Query', 'Count']
        fig2 = px.bar(topics, x='Query', y='Count', title="Most Common Topics")
        st.plotly_chart(fig2)
    except Exception as e:
        st.error(f"Error generating Most Common Topics chart: {e}")

    # User satisfaction
    st.subheader("User Satisfaction")
    try:
        satisfaction = queries['satisfaction_rating'].value_counts().reset_index()
        satisfaction.columns = ['Rating', 'Count']
        fig3 = px.pie(satisfaction, values='Count', names='Rating', title="User Satisfaction")
        st.plotly_chart(fig3)
    except Exception as e:
        st.error(f"Error generating User Satisfaction chart: {e}")

# Run Flask app and Streamlit dashboard
if __name__ == '__main__':
    # Install dependencies
    install_dependencies()

    # Initialize the database
    init_db()

    # Start Flask app in a separate thread
    import threading
    flask_thread = threading.Thread(target=app.run, kwargs={'debug': False})
    flask_thread.daemon = True
    flask_thread.start()

    # Run Streamlit dashboard
    run_dashboard()
